## practice using tensorflow

In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
print('tensorflow version:', tf.__version__)

tensorflow version: 2.9.1


In [23]:
mnist = tf.keras.datasets.mnist

In [24]:
type(mnist)

module

In [25]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [26]:
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [27]:
# add a channel dimension
x_train = x_train[..., tf.newaxis].astype('float32')
x_test = x_test[..., tf.newaxis].astype('float32')

In [28]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [29]:
type(x_train)

numpy.ndarray

use tf.data to batch and shuffle the dataset

In [30]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [31]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

build the tf.keras model using the keras model subclassing API

In [32]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [33]:
model = MyModel()

choose an optimizer and loss function for training

In [34]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

select metrics to measure the loss and accuracy of the model. these metrics accumulate the values over epochs and then print the overall result.

In [35]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

use tf.GradientTape to train the model

In [36]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training vs inference (eg dropout)
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

test the model

In [37]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training and inference (eg. dropout)
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [38]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    print(f'epoch {epoch+1}, '
         f'loss: {train_loss.result()}, '
         f'accuracy: {train_accuracy.result() * 100}, '
         f'test loss: {test_loss.result()}, '
         f'test accuracy: {test_accuracy.result() * 100}')

epoch 1, loss: 0.1400424689054489, accuracy: 95.82499694824219, test loss: 0.06122135743498802, test accuracy: 97.89999389648438
epoch 2, loss: 0.04352374002337456, accuracy: 98.67666625976562, test loss: 0.04988901689648628, test accuracy: 98.25999450683594
epoch 3, loss: 0.023122772574424744, accuracy: 99.288330078125, test loss: 0.05565894395112991, test accuracy: 98.2699966430664
epoch 4, loss: 0.014392843469977379, accuracy: 99.52166748046875, test loss: 0.07344220578670502, test accuracy: 97.97999572753906
epoch 5, loss: 0.011853854171931744, accuracy: 99.6383285522461, test loss: 0.07562345266342163, test accuracy: 98.0199966430664


### appendix

In [39]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           multiple                  320       
                                                                 
 flatten_1 (Flatten)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  2769024   
                                                                 
 dense_3 (Dense)             multiple                  1290      
                                                                 
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________
